In [40]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sqlalchemy import create_engine
import sqlalchemy.dialects
from sqlalchemy.dialects import registry
from sqlalchemy.orm import sessionmaker
import seaborn as sns

In [41]:
# Define and import from CME ftp
file_name = "ftp://ftp.cmegroup.com/pub/settle/nymex_option.csv"

In [42]:
# Create Dataframe
df = pd.read_csv(file_name, index_col=False)

# Select Columns
df.drop (["CONTRACT DAY", "CONTRACT", "OPEN", "HIGH", "PRODUCT DESCRIPTION", "HIGH AB INDICATOR", 
            "LOW", "LOW AB INDICATOR", "LAST", "LAST AB INDICATOR"
            ], axis = 1, inplace = True)

df_opts = pd.DataFrame (df,columns = ["PRODUCT SYMBOL", "CONTRACT MONTH", "CONTRACT YEAR", "PUT/CALL", "STRIKE", "SETTLE", "PT CHG", "EST. VOL", "PRIOR SETTLE", "PRIOR VOL", "PRIOR INT", "TRADEDATE"])

# Remove display delimiters
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
pd.set_option("display.width", None)
pd.set_option("display.max_colwidth", None)

# Convert NaNs to numeric format
df_opts["EST. VOL"] = pd.to_numeric(df_opts["EST. VOL"], errors="coerce")
df_opts["PT CHG"] = pd.to_numeric(df_opts["PT CHG"], errors="coerce")
df_opts["PRIOR VOL"] = pd.to_numeric(df_opts["PRIOR VOL"], errors="coerce")
df_opts["PRIOR INT"] = pd.to_numeric(df_opts["PRIOR INT"], errors="coerce")

# Drop Null numerics
df_opts = df_opts.dropna(subset=["EST. VOL"])
df_opts = df_opts.dropna(subset=["PRIOR VOL"])
df_opts = df_opts.dropna(subset=["PRIOR INT"])

# Convert float to int
df_opts["EST. VOL"] = df_opts["EST. VOL"].astype(int)
df_opts["PRIOR VOL"] = df_opts["PRIOR VOL"].astype(int)
df_opts["PRIOR INT"] = df_opts["PRIOR INT"].astype(int)

# Parse product symbol to LN (European Gas Options)
df1 = df_opts[df_opts["PRODUCT SYMBOL"].str.contains("LN")]

# Remove empty estimated volume rows
df1.loc[df1["EST. VOL"] > 0]

# Return estimated volume in descending max to min order
df1.sort_values (by = ["EST. VOL"], inplace = True, ascending = False)
df1.head()

,PRODUCT SYMBOL,CONTRACT MONTH,CONTRACT YEAR,PUT/CALL,STRIKE,SETTLE,PT CHG,EST. VOL,PRIOR SETTLE,PRIOR VOL,PRIOR INT,TRADEDATE
20417,LN,12,2020,C,3.00,0.0101,-0.0046,3862,0.0147,3990,10771,11/17/2020
20738,LN,1,2021,C,3.25,0.1095,-0.0015,3405,0.1110,4800,8898,11/17/2020
20399,LN,12,2020,C,2.75,0.0549,-0.0106,3391,0.0655,331,5269,11/17/2020
22331,LN,10,2021,C,3.25,0.1703,-0.0100,3150,0.1803,1900,25995,11/17/2020
22374,LN,10,2021,P,2.50,0.1787,0.0067,3000,0.1720,50,5555,11/17/2020


In [63]:
#Total Options Volume
Total_Volume = df1 ["EST. VOL"].sum()
print ("Total Options Volume is", Total_Volume)

Total Options Volume is 100958


In [61]:
# Total Call Volume